# 目录

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## 1.分类数据（Categorical Data）

介绍Categorcial类型，将进一步介绍进行某些操作时能获得更好的性能和内存使用。

### ①背景和目标（Background and Motivation）

In [3]:
import numpy as np; import pandas as pd

values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
print(values, '\n')
pd.unique(values)
pd.value_counts(values)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object 



apple     6
orange    2
dtype: int64

In [4]:
# 使用维度表作为数据入库时的索引，并将主要观测值存储为引用维度表的整数键
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])

dim.take(values)        # 按整数展现的方式可以被称为分类或字典编码展现

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### ②pandas中的Categorical类型（Categorical Type in pandas）

In [5]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [6]:
# 将由字符串对象组成的数组转化为Categorical类型
fruit_cat = df['fruit'].astype('category')
print(fruit_cat, '\n')

c = fruit_cat.values
print(type(c), '\n')

# Categorical类有categories和codes属性
print(c.categories, '\n')
c.codes

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange'] 

<class 'pandas.core.arrays.categorical.Categorical'> 

Index(['apple', 'orange'], dtype='object') 



array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [7]:
# 从其他Python序列直接生成Categorical类
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
print(my_categories, '\n')

# 如果已经从另一个数据源获得了分类编码数据，可以使用如下方法构造函数：(有点类似于Series中的take方法)
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
print(my_cats_2, '\n')
print(my_cats_2.as_ordered(), '\n')
# 指定顺序后：
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo'] 

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz'] 

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz'] 



['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### ③使用Categorical对象进行计算（Computations with Categoricals）

In [8]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [9]:
# 计算上述数据的四分位分箱，并提取一些统计值
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
print(bins)

# 提取汇总信息
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
print(results, '\n')
results['quartile']

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']
  quartile  count       min       max
0       Q1    250 -2.949343 -0.685484
1       Q2    250 -0.683066 -0.010115
2       Q3    250 -0.010032  0.628894
3       Q4    250  0.634238  3.927528 



0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### 使用分类获得更高性能（Better performance with categoricals）

In [10]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

# 将labels转换为Categorical对象，会发现转换后明显少占用了很多内存
categories = labels.astype('category')
print(labels.memory_usage(), '\n')
categories.memory_usage()

80000128 



10000332

In [11]:
# 分类转换是一次性开销。使用分类对象进行GroupBy操作明显更快，因为底层算法使用了基于整数代码的数组而不是字符串数组
%time _ = labels.astype('category')

CPU times: user 632 ms, sys: 39 ms, total: 671 ms
Wall time: 854 ms


### ④分类方法（Categorical Methods）

<font color='red'>pandas中Series的分类方法</font>

|方法|描述|
|:--:|:--:|
|add_categories|将新的类别（未使用过的）添加到已有类别的列表|
|as_ordered|对类别进行排序|
|as_unordered|使类别无序|
|remove_categories|去除类别，将被移除的值置为null|
|remove_unused_categories|去除所有没有出现在数据中的类别|
|rename_categories|使用新的类别名称代替现有的类别，不会改变类别的数量|
|reorder_categories|与rename_categories类似，但结果是经过排序的类别|
|set_categories|用指定的一组新类别替换现有的类别，可以添加或删除类别|

In [12]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [13]:
# 特殊属性cat提供了对分类方法的访问
cat_s.cat.codes
cat_s.cat.categories

# 使用下列方法来改变类别，看起来数据未变，但其实都反映在操作中
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
print(cat_s2, '\n')

print(cat_s.value_counts(), '\n')
cat_s2.value_counts()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e'] 

a    2
b    2
c    2
d    2
dtype: int64 



a    2
b    2
c    2
d    2
e    0
dtype: int64

In [14]:
# 除去未观察到的类别
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
print(cat_s3, '\n')
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd'] 



0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

#### 创建用于建模的虚拟变量（Creating dummy variables for modeling）

虚拟变量，也称*one-hot*编码

In [15]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(cat_s)


,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 2.高阶GroupBy应用（Advanced GroupBy Use）

### ①分组转换和“展开”GroupBy（Group Transforms and "Unwrapped" GroupBys）

- `apply`

- `transform`

    - 产生一个标量值，并广播到各分组的尺寸数据中

    - 产生一个与输入分组尺寸相同的对象

    - 不可改变输入

In [16]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [17]:
g = df.groupby('key').value
print(g.mean(), '\n')

g.transform(lambda x: x.mean())

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64 



0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [18]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [19]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [20]:
def normalize(x):
    return (x - x.mean()) / x.std()

# 内建的聚合函数如'mean'或'sum'通常与transform一起使用时会存在一个“快速通过”，因此速度要比apply函数快
print(g.transform(normalize), '\n')
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64 



0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [21]:
g.transform('mean')
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### ②分组的时间重新采样（Grouped Time Resampling）

In [22]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [23]:
# 按'time'进行索引，然后采样
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [24]:
from pandas.core import resample

# 假设增加了一个键值
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:7]
# 重新采样
# time_key = pd.TimeGrouper('5min')     # 报错无法调用
# resampled = (df2.set_index('time')
#              .groupby(['key', time_key])
#              .sum())
# resampled
# resampled.reset_index()

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


## 3.方法链技术（Techniques for Method Chaining）

```python
# 创建了许多未使用的临时变量
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)
```

```python
# 原位赋值可能较assign而言会更快，但assign可以实现更快的工具链
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

```python
df = load_data()
df2 = df[df['col2'] < 0]
```

```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

### pipe方法

```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

```python
# f(df)和df.pipe(f)是等价的，但pipe可实现连续调用
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```